In [2]:
import sys
sys.path.append('../../bstadt/NeuroDataResource')
sys.path.append('../../bstadt/Util')

import numpy as np
from NeuroDataResource import NeuroDataResource

In [3]:
with open('../data/api_key.txt', 'r') as f:
    api_key = f.readline()

In [4]:
channel_list = [
    {'name': 'annotation', 'dtype': 'uint64'},
    {'name': 'DAPI1st', 'dtype': 'uint8'},
    {'name': 'DAPI2nd', 'dtype': 'uint8'},
    {'name': 'DAPI3rd', 'dtype': 'uint8'},
    {'name': 'EM25K', 'dtype': 'uint8'},
    {'name': 'EM2k', 'dtype': 'uint8'},
    {'name': 'GABA488', 'dtype': 'uint8'},
    {'name': 'GAD647', 'dtype': 'uint8'},
    {'name': 'gephyrin594', 'dtype': 'uint8'},
    {'name': 'GS594', 'dtype': 'uint8'},
    {'name': 'MBP488', 'dtype': 'uint8'},
    {'name': 'NR1594', 'dtype': 'uint8'},
    {'name': 'PSD95_488', 'dtype': 'uint8'},
    {'name': 'Synapsin647', 'dtype': 'uint8'},
    {'name': 'VGluT1_647', 'dtype': 'uint8'}
]

In [5]:
resource = NeuroDataResource('api.boss.neurodata.io',
                               api_key,
                               'collman',
                               'collman15v2',
                               channel_list)

## Overview

When requesting data from Boss, there is some size at which the returned np.array is in float64 dtype instead of the predefined dtype. 
1. Examine whether data returned in float64 is equivalent to the defined dtype data.
2. Examine at what volume size Boss starts to return nd.array in float64 rather than the predefined dtype.

In [6]:
# Test using PSD95_488 channel, which is uint8.
img1 = resource.get_cutout('PSD95_488', [0, 4], [0, 4000], [0, 4000]) # returns uint8
img2 = resource.get_cutout('PSD95_488', [0, 4], [0, 4000], [0, 5000]) # returns float64

print(type(img1[0][0][0]))
print(type(img2[0][0][0]))

<class 'numpy.uint8'>
<class 'numpy.float64'>


In [7]:
img3 = img2.astype('uint8')

In [8]:
# Returned data are identical
print(np.array_equal(img1, img2[:, :, 0:4000]))
print(np.array_equal(img1, img3[:, :, 0:4000]))

True
True


In [9]:
# Test using EM25K channel, which is uint8.
img4 = resource.get_cutout('EM25K', [0, 4], [0, 4000], [0, 4000])
img5 = resource.get_cutout('EM25K', [0, 4], [0, 4000], [0, 5000])

print(type(img4[0][0][0]))
print(type(img5[0][0][0]))

<class 'numpy.uint8'>
<class 'numpy.float64'>


In [10]:
img6 = img5.astype('uint8')

In [11]:
print(np.array_equal(img4, img5[:, :, 0:4000]))
print(np.array_equal(img4, img6[:, :, 0:4000]))

True
True


### Testing at what size NeuroDataResouce starts returning float64 instead of defined dtype.

In [12]:
def granular_analysis(z_start, y_start, x_start, interval):
    for z in range(z_start, 28):
        for y in range(y_start, 4500, interval):
            for x in range(x_start, 6306, interval):
                img = resource.get_cutout('PSD95_488', [0, z], [0, y], [0, x])
                dtype = type(img[0][0][0])
                if dtype == type(np.float64()):
                    print(z, y, x)
                    return

In [13]:
granular_analysis(4, 2560, 5120, 128)

4 2688 6272


In [14]:
granular_analysis(4, 2688, 6272 - 128, 4)

4 2688 6244


In [15]:
granular_analysis(4, 2688, 6244 - 4, 1)

4 2688 6242


Confirm whether the dimensions (4, 2688, 6242) is the point at which Boss returns float64.

In [16]:
type(resource.get_cutout('PSD95_488', [0, 4], [0, 2688], [0, 6241])[0][0][0])

numpy.uint8

In [17]:
type(resource.get_cutout('PSD95_488', [0, 4], [0, 2688], [0, 6242])[0][0][0])

numpy.float64